In [7]:
import pandas as pd

# JSONL 파일 경로
jsonl_file_path = 'C:/Users/wjdrb/vscode_code/MathBridge_new/data/equations_with_context_math_serial-0626-0007.jsonl'

# JSONL 파일을 DataFrame으로 불러오기
try:
    with open(jsonl_file_path, 'r', encoding='utf-8') as file:
        df = pd.read_json(file, lines=True)
    
except ValueError as e:
    print(f"JSONL 파일을 읽는 중 오류 발생: {e}")
except FileNotFoundError:
    print("JSONL 파일을 찾을 수 없습니다. 파일 경로를 확인하세요.")


$a-q_{1}b=1\cdot a+(-q_{1})\cdot b=r_{1}$

In [8]:
df

,context_before,equation,context_after,paper_number
0,rnamecolors begindocument maketitle beginabstr...,"$(k,\ell)$",pebble game with and use it to obtain a charac...,704.0002
1,use it to obtain a characterization of the fam...,"$(k,\ell)$",sparse graphs and algorithmic solutions to a f...,704.0002
2,present a new decomposition that certifies spa...,"$(k,\ell)$",pebble game with Our work also exposes connect...,704.0002
3,and preliminaries The focus of this paper is d...,"$(k,\ell)$",sparse graphs into edgedisjoint subgraphs that...,704.0002
4,a possibly with We say that a graph is bf,"$(k,\ell)$",sparse if no subset of n vertices spans more than,704.0002
...,...,...,...,...
2035419,diagonal one can therefore assume that there e...,$n_k$,such that unki mbox converges in L2Ui mbox for...,709.3736
2035420,therefore assume that there exists a subsequen...,$$\nu_{n_k}^i \mbox{ converges in } L^2(U_i) \...,the sequence unk sumi in I unki is convergent in,709.3736
2035421,converges in L2Ui mbox for all i in the sequence,$u_{n_k} = \sum_{i \in I} u_{n_k}^i$,is convergent in proofend sectionThe case of t...,709.3736
2035422,the sequence unk sumi in I unki is convergent in,$L^2(O)$,proofend sectionThe case of the operator tilde...,709.3736


In [9]:
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
from io import BytesIO
from concurrent.futures import ThreadPoolExecutor, as_completed


# Chrome WebDriver 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

def render_latex_to_images(latex_list, output_paths):
    try:
        # Chrome WebDriver 인스턴스 생성
        driver = webdriver.Chrome(options=chrome_options)

        # HTML 템플릿 생성
        latex_items = "".join([f'<div id="equation{i}">${latex}$</div>' for i, latex in enumerate(latex_list)])
        html_template = f"""
        <html>
        <head>
        <script type="text/x-mathjax-config">
          MathJax.Hub.Config({{
            tex2jax: {{inlineMath: [['$','$'], ['\\(','\\)']]}}
          }});
        </script>
        <script type="text/javascript" async
          src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/MathJax.js?config=TeX-AMS_CHTML">
        </script>
        </head>
        <body>
        {latex_items}
        </body>
        </html>
        """

        # HTML 파일 생성
        html_path = 'temp.html'
        with open(html_path, 'w') as f:
            f.write(html_template)

        # HTML 파일 로드
        driver.get('file://' + os.path.abspath(html_path))

        # MathJax 렌더링 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, f"equation{len(latex_list) - 1}"))
        )

        results = []
        for i, output_path in enumerate(output_paths):
            is_error = driver.execute_script(f"""
                const element = document.getElementById('equation{i}');
                return element.textContent.includes('Math Processing Error');
            """)

            if is_error:
                results.append("error")
            else:
                element = driver.find_element(By.ID, f"equation{i}")
                png = element.screenshot_as_png
                image = Image.open(BytesIO(png))
                image.save(output_path)
                results.append(output_path)

        driver.quit()
        return results

    except Exception as e:
        driver.quit()
        return ["error"] * len(latex_list)

# 수식 이미지를 저장할 폴더 생성
output_dir = 'equation_images'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 병렬 처리 함수
def process_equations(equations, start_idx):
    output_paths = [os.path.join(output_dir, f"equation_{start_idx + i}.png") for i in range(len(equations))]
    return render_latex_to_images(equations, output_paths)

# 병렬 처리 실행
batch_size = 100
max_workers = 20
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = []
    for i in range(0, len(df), batch_size):
        batch = df['equation'][i:i + batch_size].tolist()
        futures.append(executor.submit(process_equations, batch, i))

    results = []
    for future in as_completed(futures):
        results.extend(future.result())

# 결과를 DataFrame에 추가
df['image_path'] = results



Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)
